# Import Library

In [20]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import MobileNet_V3_Small_Weights
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, r2_score
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import joblib
from sklearn.model_selection import train_test_split

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device} as device.")

Using cpu as device.


# Function Library

In [10]:
class BMIDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform
        
        # Remove any rows without image files
        self.df['full_path'] = self.df['name'].apply(lambda x: os.path.join(image_dir, x))
        self.df = self.df[self.df['full_path'].apply(os.path.exists)].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_path']).convert('RGB')

        if self.transform:
            img = self.transform(img)

        label = row['bmi']
        return img, label, idx 

# Model Loading

In [4]:
model = mobilenet_v3_small(weights=None)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[0].in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
)
model.load_state_dict(torch.load("cnn_model_final.pt"))
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [6]:
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [14]:
# Transform
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Dataset (adjust path to Colab)
dataset = BMIDataset(
    csv_path='landmark_features.csv',
    image_dir='../data/BMI/Images',
    transform=img_transform
)

# Split + Loaders
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [15]:
all_preds = []

with torch.no_grad():
    for imgs, _, _ in val_loader:
        imgs = imgs.to(device)
        preds = model(imgs).cpu().numpy().flatten()
        all_preds.extend(preds)

In [18]:
df = pd.read_csv("landmark_features.csv")
X = df.drop(columns=["bmi", "name"]).values
y = df["bmi"].values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
scaler = joblib.load("mlp_scaler.pkl")
y_test_scaled = scaler.transform(y_test.reshape(-1, 1)).ravel()

In [24]:
# Load saved model
mlp_model = joblib.load("mlp_landmark_model.pkl")

# Predict
y_pred_scaled = mlp_model.predict(X_test)

In [25]:
mlp_preds = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

In [26]:
y_true = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

In [27]:
ensemble_preds = 0.5 * np.array(all_preds) + 0.5 * np.array(mlp_preds)

In [28]:
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr

mae = mean_absolute_error(y_true, ensemble_preds)
r2 = r2_score(y_true, ensemble_preds)
r, _ = pearsonr(y_true, ensemble_preds)

print(f"Ensemble MAE: {mae:.2f}")
print(f"Ensemble R²: {r2:.3f}")
print(f"Ensemble Pearson r: {r:.3f}")

Ensemble MAE: 283.44
Ensemble R²: -17.249
Ensemble Pearson r: 0.274


In [29]:
for w in np.linspace(0, 1, 11):  # Try weights from 0.0 to 1.0
    blended = w * np.array(all_preds) + (1 - w) * np.array(mlp_preds)
    r, _ = pearsonr(y_true, blended)
    print(f"Weight CNN: {w:.1f} | Pearson r: {r:.3f}")


Weight CNN: 0.0 | Pearson r: 0.270
Weight CNN: 0.1 | Pearson r: 0.270
Weight CNN: 0.2 | Pearson r: 0.271
Weight CNN: 0.3 | Pearson r: 0.272
Weight CNN: 0.4 | Pearson r: 0.273
Weight CNN: 0.5 | Pearson r: 0.274
Weight CNN: 0.6 | Pearson r: 0.273
Weight CNN: 0.7 | Pearson r: 0.268
Weight CNN: 0.8 | Pearson r: 0.249
Weight CNN: 0.9 | Pearson r: 0.185
Weight CNN: 1.0 | Pearson r: 0.041
